<a href="https://colab.research.google.com/github/Fecry/MetNumUN2024I/blob/main/Lab7/week_4_iterative_linalg_Jacobi_Seidel_minres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grupo 22

#### Integrantes:


*   (22) Jesus Andres Inguilan Paguay
*   (42) Cristian Felipe Ramirez Montenegro
*   (55) Victor Manuel Torres Alonso


# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

In [3]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [4]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [6]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction.
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [7]:
# ... ENTER YOUR CODE HERE ...
def jacobi(A, b, num_it=1000):
    n = A.shape[0]
    diag_A = np.diag(A)
    B = np.zeros_like(A)

    for i in range(n):
        for j in range(n):
            if i != j:
                B[i, j] = -A[i, j] / diag_A[i]

    c = b / diag_A
    norm = np.linalg.norm(B)

    x0 = np.ones(n)
    x = x0
    for _ in range(num_it):
        x = B @ x + c

    return x, norm


jacobi(A, b)

(array([ 0.03919429,  0.03780037,  0.04283232,  0.02365951,  0.05745031,
        -0.00030244, -0.00577279,  0.03177549, -0.00422849,  0.05284648]),
 0.36436161983015336)

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration.

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [8]:
# ... ENTER YOUR CODE HERE ...
def gauss_seidel(A, b, x_0, tol, max_iter):

  x = x_0.copy()

  for i in range(max_iter):

    for j in range(A.shape[0]):
      x[j] = (b[j] - np.dot(A[j, :j], x[:j])) / A[j, j]

    norm_x = np.linalg.norm(x - x_0)
    if norm_x < tol:
      break

  return x

A = np.random.rand(10, 10)
b = np.random.rand(10)
print(A)
x = gauss_seidel(A, b, np.zeros(10), 1e-6, 100)

true_x = np.linalg.solve(A, b)

error = np.linalg.norm(x - true_x)
print(error)

error_list = []
for i in range(1, 100):
  x = gauss_seidel(A, b, np.zeros(10), 1e-6, i)
  error_list.append(np.linalg.norm(x - true_x))

[[0.55023677 0.46260879 0.92634397 0.55387927 0.73113228 0.24069015
  0.36119104 0.05609323 0.79271832 0.79127   ]
 [0.13681345 0.00544233 0.48929869 0.53592179 0.68670853 0.95538054
  0.09291416 0.0638495  0.48852291 0.47054225]
 [0.9363455  0.67007034 0.04613604 0.67344904 0.35047814 0.13791389
  0.93150188 0.70896135 0.43501103 0.76591055]
 [0.75500834 0.40171163 0.14723796 0.24832737 0.54136421 0.10586887
  0.46271696 0.27254075 0.6032712  0.19446414]
 [0.32637471 0.28399311 0.12884414 0.65640229 0.19049124 0.16125886
  0.96697425 0.32444531 0.35508147 0.31231128]
 [0.36743656 0.56110535 0.59956025 0.95062848 0.65066597 0.83885008
  0.43877938 0.38199392 0.45072847 0.46228793]
 [0.44802987 0.26986376 0.10548617 0.9977702  0.74868178 0.3301447
  0.50916151 0.65463998 0.15878148 0.17916064]
 [0.22517771 0.67660532 0.15589731 0.28586035 0.50122307 0.09638936
  0.12886134 0.19488287 0.48152421 0.10062812]
 [0.2953252  0.16994135 0.8739273  0.49631712 0.24641884 0.85717588
  0.03248567 

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...